In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import CacheBackedEmbeddings, OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

llm = ChatOpenAI(
	temperature=0.1,
)

memory = ConversationBufferMemory()

cache_dir = LocalFileStore("../.cache/")
splitter = CharacterTextSplitter.from_tiktoken_encoder(
	separator="\n",
	chunk_size=600,
	chunk_overlap=50,
)
loader = UnstructuredFileLoader("../files/chapter_three.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vector_store = FAISS.from_documents(docs, cached_embeddings)
retriever = vector_store.as_retriever()


### 문서를 처리하기 위한 프롬프트와 체인.
map_doc_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"""
		아래의 매우 긴 문서에서 질문에 대한 응답과 관련된 부분을 추출합니다.
		만약 관련있는 문장이 없다면, '' 를 리턴하세요.
		------
		{context}	
		"""
	),
	(
		"human",
		"{question}"
	),
])
map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
	documents = inputs["documents"]
	question = inputs["question"]

	return "\n\n".join(
		map_doc_chain.invoke({
			"context": doc.page_content,
			"question": question
		}).content
		for doc in documents
	)

map_chain = { "documents": retriever, "question": RunnablePassthrough() } | RunnableLambda(map_docs)


### 요약 정리된 문서에서 질문을 처리하는 프롬프트와 체인
final_prompt = ChatPromptTemplate.from_messages([
	(
		"system",
		"""
		아래의 요약 정리된 문서를 참고해서 응답을 작성합니다.
		만약 답을 모르겠다면 모르겠다고 응답합니다.
		억지로 만들어내지 마세요.
		------
		{context}
		"""
	),
	(
		"human",
		"{question}"
	),
])
chain = { "context": map_chain, "question": RunnablePassthrough() } | final_prompt | llm

def invoke_chain(question):
	result = chain.invoke(question)
	memory.save_context(
		{"input": question},
		{"output": result.content}
	)
	print(result.content)


In [ ]:
invoke_chain("Aaronson 은 유죄인가요?")

아론슨이 유죄인지에 대한 정보는 제공되지 않았습니다. 위의 문서에서는 아론슨에 대한 언급이 없습니다.


In [19]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

그가 테이블에 어떤 메시지를 썼다는 내용은 문서에서 언급되지 않았습니다.


In [20]:
invoke_chain("Julia 는 누구인가요?")

Julia는 주인공인 Winston와 함께 등장하는 여성 캐릭터로, Winston과 사랑에 빠지는 캐릭터입니다. Winston가 절망 속에서 자신을 대신해 고통을 받아줄 수 있는 유일한 사람으로 Julia를 지칭하며 강한 감정을 느낍니다.


In [21]:
memory.load_memory_variables({})['history']

'Human: Aaronson 은 유죄인가요?\nAI: 아론슨이 유죄인지에 대한 정보는 제공되지 않았습니다. 위의 문서에서는 아론슨에 대한 언급이 없습니다.\nHuman: 그가 테이블에 어떤 메시지를 썼나요?\nAI: 그가 테이블에 어떤 메시지를 썼다는 내용은 문서에서 언급되지 않았습니다.\nHuman: Julia 는 누구인가요?\nAI: Julia는 주인공인 Winston와 함께 등장하는 여성 캐릭터로, Winston과 사랑에 빠지는 캐릭터입니다. Winston가 절망 속에서 자신을 대신해 고통을 받아줄 수 있는 유일한 사람으로 Julia를 지칭하며 강한 감정을 느낍니다.'